In [1]:
import numpy as np
with open('reviews.txt','r') as f:
    reviews = f.read()
with open('labels.txt','r') as f:
    labels = f.read()

In [2]:
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [3]:
from  string import punctuation

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

words = all_text.split()


In [4]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [5]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}

reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()]) 

In [6]:
print('Unique words: ', len((vocab_to_int)))
print()
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  53766

Tokenized review: 
 [[14010, 319, 6, 3, 1118, 202, 8, 2214, 32, 1, 168, 57, 15, 49, 78, 5681, 44, 420, 114, 140, 15, 4026, 61, 147, 9, 1, 4783, 5682, 481, 70, 5, 258, 12, 14010, 319, 13, 2263, 6, 73, 2507, 5, 593, 74, 6, 4026, 1, 17942, 5, 1889, 9261, 1, 5969, 1557, 36, 52, 65, 207, 145, 67, 1240, 4026, 19990, 1, 26843, 4, 1, 225, 881, 31, 3159, 70, 4, 1, 6119, 10, 703, 2, 67, 1557, 55, 10, 212, 1, 353, 9, 64, 3, 1390, 3633, 756, 5, 3542, 177, 1, 420, 10, 1187, 16357, 32, 319, 3, 347, 334, 3083, 10, 143, 129, 5, 7036, 30, 4, 130, 4026, 1390, 2452, 5, 14010, 319, 10, 525, 12, 110, 1403, 4, 61, 589, 102, 12, 14010, 319, 6, 234, 4164, 48, 3, 2453, 12, 8, 230, 21]]


In [7]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [8]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2514


In [9]:
 print('Number of reviews before removing outliers: ', len(reviews_ints))

non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  12110
Number of reviews after removing outliers:  12110


In [10]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
    for i,row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [11]:
seq_length = 200
features = pad_features(reviews_ints, seq_length=seq_length)

assert len(features) == len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

print(features[:3,:20])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [16359    43 33699    15   740 13113  3677    47    75    35  1959    16
    147    19   111     3  1279     5   336   146]]


In [12]:
split_frac = 0.8
split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y , remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print("\t\t Feature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), "\nValidation set: \t{}".format(val_x.shape), "\nTest set: \t\t{}".format(test_x.shape))

		 Feature Shapes:
Train set: 		(9688, 200) 
Validation set: 	(1211, 200) 
Test set: 		(1211, 200)


In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data =  TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=50)
test_loader = DataLoader(test_data, shuffle=True, batch_size=50)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=50)


In [14]:
dataiter = iter(valid_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size())
print('Sample input \n', sample_x)
print()
print('Sample label size: ', sample_y.size())
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input 
 tensor([[ 624,  999,  562,  ...,   24,   15, 1302],
        [   0,    0,    0,  ...,   57,    5,  105],
        [  10,  143,   24,  ...,   41,   13,   53],
        ...,
        [   0,    0,    0,  ...,    1,  658,  114],
        [   0,    0,    0,  ...,   21,  105,   11],
        [  39,   10,  301,  ...,  716,  281,   22]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1], dtype=torch.int32)


In [15]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN,self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out[:, -1, : ]
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [16]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentRNN(
  (embedding): Embedding(53767, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [17]:
lr = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [18]:
epochs = 4
batch_size = 50
counter = 0
print_every = 100

clip = 5

net.train()

for e in range(epochs):
    
    h = net.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        if(len(inputs)<50):
                    break
        counter += 1
        
        h = tuple([each.data for each in h])
        
        net.zero_grad()
        output, h = net(inputs, h)
        #print(len(h), len(output))
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        if counter % print_every == 0:
            val_h = net.init_hidden(50)
            val_losses = []
            net.eval()
            for inp, labels in valid_loader:
                if(len(inp)<50):
                    break
                val_h = tuple([each.data for each in val_h])
                
                output, val_h = net(inp, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                val_losses.append(val_loss.item())
            net.train()
            print("Ëpoch: {}/{}...".format(e+1, epochs),
                 "Step: {}..".format(counter),
                 "Loss: {:.6f}...".format(loss.item()),
                 "Val Loss: {:.6f}".format(np.mean(val_losses)))


Ëpoch: 1/4... Step: 100.. Loss: 0.608746... Val Loss: 0.651273
Ëpoch: 2/4... Step: 200.. Loss: 0.686467... Val Loss: 0.693858
Ëpoch: 2/4... Step: 300.. Loss: 0.635827... Val Loss: 0.651590
Ëpoch: 3/4... Step: 400.. Loss: 0.522323... Val Loss: 0.624199
Ëpoch: 3/4... Step: 500.. Loss: 0.797201... Val Loss: 0.798805
Ëpoch: 4/4... Step: 600.. Loss: 0.217707... Val Loss: 0.682495
Ëpoch: 4/4... Step: 700.. Loss: 0.297671... Val Loss: 0.655398


In [20]:
test_losses = []
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:
    if(len(inputs)<50):
        break
    h = tuple([each.data for each in h])
    output, h = net(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze())
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
print("Test loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.756
Test accuracy: 0.724


In [33]:
test_review_neg = 'The Best movie I have seen; acting was terrible. This movie had good acting and the dialogue was fantastic.'

In [34]:
from string import punctuation
def tokenize_review(test_review):
    test_review = test_review.lower()
    test_text = ''.join([c for c in test_review if c not in punctuation])
    
    test_words = test_text.split()
    
    test_ints = []
    test_ints.append([vocab_to_int.get(word, 0 ) for word in test_words])
    
    return test_ints

test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 116, 18, 10, 28, 109, 115, 14, 372, 11, 18, 68, 51, 115, 2, 1, 403, 14, 710]]


In [35]:
seq_length = 200
features = pad_features(test_ints, seq_length)
print(features)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   1 116  18  10  28 109 115  14 372  11  18  68  51 115   2   1 403
   14 710]]


In [36]:
def predict(net , test_review, sequence_length=200):
    net.eval()
    test_ints = tokenize_review(test_review)
    
    seq_length = sequence_length
    
    features = pad_features(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    
    h = net.init_hidden(batch_size)
    
    output, h  = net(feature_tensor, h)
    
    pred = torch.round(output.squeeze())
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    if(pred.item()==1):
        print("Positive review")
    else:
        print("Negative review")

In [37]:
predict(net, test_review_neg, sequence_length=200)

Prediction value, pre-rounding: 0.504346
Positive review
